<a href="https://colab.research.google.com/github/loren198/RCPSP_1/blob/master/ProcureDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install openai --upgrade
#%pip install dotenv
%pip install datetime



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 12.4 MB/s eta 0:00:00


In [2]:
#import os,dotenv
#from dotenv import load_dotenv
import openai

In [43]:
# For local key
# Import Key and Base from .env
# load_dotenv("secrets.env")
# openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.organization = os.getenv("OPENAI_ORGANIZATION")

# param for have result more deterministic ( 0) or creative (1) you could have step for experimentations like 0.5
temperature = 0.2

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')
# openai.organization = userdata.get('OPENAI_ORGANIZATION')



# Verify if loading is correct
# print("OpenAI organization : " + openai.organization)

In [ ]:
#Import multiple files
import glob
folder_path = "/content/Documentation"

# Use glob to get a list of files in the folder
files = glob.glob(folder_path + '/*')

# Create a list to store file IDs
file_ids = []

# Iterate through files in the folder and upload each file
for file_path in files:
    # Upload each file with "assistants" purpose
    file = openai.files.create(
        file=open(file_path, "rb"),
        purpose='assistants'
    )

    # Append the file ID to the list
    file_ids.append(file.id)

    # Optionally print information about the uploaded file
    print(f"Uploaded file: {file_path}, ID: {file.id}")

In [ ]:
#create assistant
assistant = openai.beta.assistants.create(
  instructions="Select the best software option based only on the documents provided and user needs. The answer must be maximum 3 bullets, each with one software option, ordered by relevance. For each bullet, provide the software commercial name, price range and a 1-2 line reasoning for your selection",
  model="gpt-4-turbo-preview",
  tools=[{"type": "retrieval"}],
  file_ids=file_ids
)

In [21]:
#if already created, use this to update

assistant = openai.beta.assistants.update(
  assistant_id="asst_OnoUijWYrtcaOLCHPbgyBQ5M",
  instructions="Select the best software option based only on the documents provided and user needs. The answer must be maximum 2 bullets, each with one software option, ordered by relevance. For each bullet, provide the software commercial name, price range and a 1-2 line reasoning for your selection"
)

In [22]:
#Initialize thread

thread = openai.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": input("What are your needs?"),
    }
  ]
)

#run question
run = openai.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)



#retrieve answer
run = openai.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

import time

while run.status !="completed":
  run = openai.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
  time.sleep (2)
  #print(run.status)



thread_messages = openai.beta.threads.messages.list(
    thread_id=thread.id)
#print(thread_messages.data[0])
for message in thread_messages.data:
    for content in message.content:
            print(content.text.value)

What are your needs?Non-sql relational database manager
Based on the documents provided:

1. **Amazon DynamoDB**
   - **Price Range:** Pricing details not explicitly provided in the document, but it operates on a pay-as-you-go pricing model typical for AWS services.
   - **Reasoning:** Amazon DynamoDB is a highly available and durable NoSQL database service that supports key-value and document data structures, offering flexibility in managing non-relational data with features like automatic scaling, ACID transactions, and global tables【11†source】.

2. **Neo4j**
   - **Price Range:** Pricing details not explicitly provided in the document, but it offers both a Community edition which is free and a paid Enterprise edition.
   - **Reasoning:** Neo4j is a graph database management system known for its efficiency in handling highly connected data. The document covers detailed introduction to graph databases, the Cypher query language, data modelling, and data importing, emphasizing its capa

In [47]:
#update assistant to respond to questions
openai.beta.assistants.update(
  assistant_id=assistant.id,
  instructions= "You are a technical expert in a software specified by the user. The user will provide the name of the software and a document with multiple questions regarding that software. Answer all user questions in the following way: first, restate in exactly the same words the user first question. Then, in a new line provide your answer to the question and cite in what page you found it. Repeat the process for all questions in the document, numbering the questions.",
  #instructions= "You are a technical expert in a software specified by the user. The user will provide the name of the software and a document with questions regarding that software. Based only on the documentation of that software that you have access to, answer the user questions in the following way: first, restate in exactly the same words the user first question. Then, in a new line provide your answer to the question and cite in what page you found it. Repeat the process for subsequent questions. If the answer is not available, just say Answer not available",
)

Assistant(id='asst_OnoUijWYrtcaOLCHPbgyBQ5M', created_at=1710565174, description=None, file_ids=['file-ZVCRJMjyaIRDaPUwK3zrs5RL', 'file-PMWYYvWidxbbAGDODbNfJ0QJ', 'file-U8wRajE4XAcQtdLBj33UzF7m'], instructions='You are a technical expert in a software specified by the user. The user will provide the name of the software and a document with multiple questions regarding that software. Answer all user questions in the following way: first, restate in exactly the same words the user first question. Then, in a new line provide your answer to the question and cite in what page you found it. Repeat the process for all questions in the document, numbering the questions.', metadata={}, model='gpt-4-turbo-preview', name=None, object='assistant', tools=[RetrievalTool(type='retrieval')])

In [50]:
# Upload a file with an "assistants" purpose
file = openai.files.create(
  file=open("/content/Neo4JSampleQ2.pdf", "rb"),
  purpose='assistants'
)

threadQ = openai.beta.threads.create()

message = openai.beta.threads.messages.create(
  thread_id=threadQ.id,
  role="user",
  content=input("What software are you looking for? ") + ". Answer the questions in the file attached.",
  file_ids=[file.id]
)

#run question
runQ = openai.beta.threads.runs.create(
  thread_id=threadQ.id,
  assistant_id=assistant.id
)


#retrieve answer
runQ = openai.beta.threads.runs.retrieve(
  thread_id=threadQ.id,
  run_id=runQ.id
)

import time

#time.sleep (90)
while runQ.status !="completed":
  time.sleep (2)
  runQ = openai.beta.threads.runs.retrieve(
  thread_id=threadQ.id,
  run_id=runQ.id
)

  #print(run.status)

thread_messages = openai.beta.threads.messages.list(
    thread_id=threadQ.id)
#print(thread_messages.data[0])
for message in thread_messages.data:
    for content in message.content:
            print(content.text.value)

What software are you looking for? Neo4J
I'm unable to access the document directly due to its incompatibility with the browsing tool. Could you please upload the content of the document as text, or describe the questions you need assistance with? This will enable me to assist you more effectively.
Neo4J. Answer the questions in the file attached.


In [49]:
message = openai.beta.threads.messages.create(
  thread_id=threadQ.id,
  role="user",
  content="Yes you can read the file. Retry",
  file_ids=[file.id]
)

#run question
runQ = openai.beta.threads.runs.create(
  thread_id=threadQ.id,
  assistant_id=assistant.id
)


#retrieve answer
runQ = openai.beta.threads.runs.retrieve(
  thread_id=threadQ.id,
  run_id=runQ.id
)

while runQ.status !="completed":
  time.sleep (2)
  runQ = openai.beta.threads.runs.retrieve(
  thread_id=threadQ.id,
  run_id=runQ.id
)
  #print(run.status)


thread_messages = openai.beta.threads.messages.list(
    thread_id=threadQ.id)
#print(thread_messages.data[0])
for message in thread_messages.data:
    for content in message.content:
            print(content.text.value)

1. What are the Python integration options available?

Neo4J can be integrated with Python using the Neo4j Python driver, which is officially supported by Neo4J. This driver allows for a direct connection to the database using Cypher, Neo4J's query language. Additionally, Py2neo, a third-party library, provides a more Pythonic way to work with Neo4J, including OGM (Object Graph Mapper) support for modeling and interacting with the database in a more object-oriented manner.

2. Can you provide details about the performance benchmarks for typical use cases especially for large datasets and complex queries?

Details about performance benchmarks for Neo4J, particularly for large datasets and complex queries, are usually found in the Neo4J whitepapers and performance analysis reports. These documents provide insights into various performance metrics like query execution time, read/write throughput, and scalability under different workloads. Performance can highly depend on factors such as t